In [560]:
import pandas as pd
import numpy as np
import math

df = pd.read_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\BGB-16673-102_V1.0.xlsx', sheet_name='CheckSteps')
df

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,SVSTAT,NaN,SV,SVSTAT,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13618,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [561]:
pd.set_option('display.max_rows', 3)
pd.set_option('display.max_columns', 30)

#### Data clean before processing data

In [562]:
# Exclude RSG parts
df = df[~df['CheckName'].str.contains('RSG')]
df

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,SVSTAT,NaN,SV,SVSTAT,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13618,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [563]:
# list out forms that're repeated: CM_PST | AE_MIX | CM_POST | etc...
def isFormRpt(x):
    if x == 'CM_PST':
        result = 1
    elif x == 'AE_MIX':
        result = 1
    elif x == 'CM_POST':
        result = 1
    else:
        result = 0
    return result

df['Is_Form_Repeated'] = df['FormOID'].apply(isFormRpt)
#df.loc[:,'Is_Form_Repeated'] = df['FormOID'].apply(isFormRpt)
df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,SVSTAT,NaN,SV,SVSTAT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13618,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [564]:
# Aggregate: sum Is_Form_Repeated Comumn values for each Check
Is_Form_Repeated_Sum = df.groupby('CheckName').sum()
Is_Form_Repeated_Sum = pd.DataFrame(Is_Form_Repeated_Sum,columns=['Is_Form_Repeated'])
Is_Form_Repeated_Sum = Is_Form_Repeated_Sum.rename(columns={'Is_Form_Repeated':'Is_Form_Repeated_Sum'})
Is_Form_Repeated_Sum

,Is_Form_Repeated_Sum
CheckName,
CF_DER_LABAGE_A,0
...,...
Update_AllInstanceNames_Empty_CF,0


In [565]:
Is_Form_Repeated_Sum.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\Is_Form_Repeated_Sum.xlsx')

#### Join: primary table(dataframe) + Is_Form_Repeated_Sum

In [566]:
# Join Is_Form_Repeated_Sum to df dataframe
join = pd.merge(df,Is_Form_Repeated_Sum,how='left',on='CheckName')
join

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum
0,DYN_SV_SET_SVDELYN_VISIBLE,1,NaN,NaN,CodedValue,SVSTAT,NaN,SV,SVSTAT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13364,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [567]:
join.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\join.xlsx',index=False)

In [568]:
#join = join.drop(columns=['FormRpt_x'])
#join

In [569]:
join = join[join['Is_Form_Repeated_Sum'] > 0]
join

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum
65,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13364,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [570]:
# AE_MIX    AE_MIX
# AE_MIX => AE_MIX => AE_MIX => AEMIX|DS_EOT
# NaN       DS_EOT    DS_EOT
# DS_EOT

combine_FormOID = join[~join['FormOID'].isna()]

combine_FormOID = combine_FormOID.drop_duplicates(subset=['CheckName','FormOID'])

combine_FormOID['FormOID'] = combine_FormOID['FormOID'].astype(str)

combine_FormOID = combine_FormOID.groupby('CheckName')['FormOID'].apply('|'.join).reset_index()

combine_FormOID

,CheckName,FormOID
0,Chk_AE_MIX_AEACNOTH_CM1_01_a_CF,AE_MIX
...,...,...
102,Derive_MinDate_Z_CMEDAT4_b_CF,CM_POST


In [571]:
combine_FormOID.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\combine_FormOID.xlsx',index=False)

#### Join_1: Join(df + Is_Form_Repeated_Sum) + combine_FormOID

In [572]:
join_1 = pd.merge(join,combine_FormOID,how='left',on='CheckName')
join_1

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,CM_PST|MH_DHH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST


In [573]:
join_1.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\join_1.xlsx',index=False)

#### Check If FormRptNum Is Entered

In [574]:
# Filter as below example:
# AE_MIX            
# AE_MIX|DS_EOT => AE_MIX|DS_EOT

join_1_x = join_1[join_1['FormOID_y'].str.contains(r'[@#&$%+-/|*]')]
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,CM_PST|MH_DHH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST|DS_EOS


In [575]:
#join_1_x['NoFormRptNumber'] = join_1_x[(~join_1_x['FormOID_x'].isna()) & (join_1_x['FormOID_x'] != 'CM_PST') & (join_1_x['FormOID_x'] != 'AE_MIX') & (join_1_x['FormOID_x'] != 'CM_POST') &
                              #(join_1_x['FormRepeatNumber'].isna())]
# -1 means FormOID is blank
# 99 menas non-relevant
# 1 means find FormRptNum not filled
# 0 means not found
# 99 menas non-relevant
def FormRptNumIsEmpty(x,y):
    if pd.isna(x):
        result = 0
    elif ~pd.isna(x) and x != 'CM_PST' and x != 'AE_MIX' and x != 'CM_POST' and pd.isna(y):
        result = 1
    elif ~pd.isna(x) and x != 'CM_PST' and x != 'AE_MIX' and x != 'CM_POST'and ~pd.isna(y):
        result = 0
    else:
        result = 0
    return result

join_1_x['FormRptNum_IsEmpty'] = join_1_x.apply(lambda x: FormRptNumIsEmpty(x['FormOID_x'], x['FormRepeatNumber']), axis=1)
join_1_x

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y,FormRptNum_IsEmpty
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,CM_PST|MH_DHH,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST|DS_EOS,0


In [576]:
join_1_x.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\FormRptNum_IsEmpty.xlsx',index=False)

In [577]:
FormRptNumIsEmpty_Sum = join_1_x.groupby('CheckName').sum()
FormRptNumIsEmpty_Sum

,StepOrdinal,RecordPosition,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormRptNum_IsEmpty
CheckName,,,,,,,
Chk_AE_MIX_AEACN_DW1_04,171,0.0,0.0,0.0,3,54,0
...,...,...,...,...,...,...,...
DYN_CM_PST_SET_PSTTYOTH_VISIBLE,91,0.0,0.0,0.0,2,26,2


In [578]:
FormRptNumIsEmpty_Sum = pd.DataFrame(FormRptNumIsEmpty_Sum,columns=['FormRptNum_IsEmpty'])
FormRptNumIsEmpty_Sum

,FormRptNum_IsEmpty
CheckName,
Chk_AE_MIX_AEACN_DW1_04,0
...,...
DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2


In [579]:
FormRptNumIsEmpty_Sum = FormRptNumIsEmpty_Sum.rename(columns={'FormRptNum_IsEmpty': 'FormRptNum_IsEmpty_Sum'})
FormRptNumIsEmpty_Sum

,FormRptNum_IsEmpty_Sum
CheckName,
Chk_AE_MIX_AEACN_DW1_04,0
...,...
DYN_CM_PST_SET_PSTTYOTH_VISIBLE,2


In [580]:
join_1_x = pd.merge(join_1_x,FormRptNumIsEmpty_Sum,how='left',on='CheckName')
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y,FormRptNum_IsEmpty,FormRptNum_IsEmpty_Sum
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,CM_PST|MH_DHH,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST|DS_EOS,0,0


In [582]:
#join_1_x = join_1_x.drop(columns=['FormRpt_y','FormRptNumIsEmpty'])
#join_1_x

In [583]:
def Is_EmptyFormRptNum_Existed(x):
    if x > 0:
        result = 'Yes'
    elif x == 0:
        result = 'No'
    else:
        result = 'n/a'
    return result

join_1_x['FormRptNum_IsEmpty_Existed'] = join_1_x['FormRptNum_IsEmpty_Sum'].apply(Is_EmptyFormRptNum_Existed)
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y,FormRptNum_IsEmpty,FormRptNum_IsEmpty_Sum,FormRptNum_IsEmpty_Existed
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,CM_PST|MH_DHH,0,2,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST|DS_EOS,0,0,No


In [584]:
join_1_x = join_1_x.drop(columns=['Is_Form_Repeated','Is_Form_Repeated_Sum','FormOID_y','FormRptNum_IsEmpty','FormRptNum_IsEmpty_Sum'])
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRptNum_IsEmpty_Existed
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Chk_CM_POST_REASEND4_01,5,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [586]:
join_1_x = join_1_x[join_1_x['FormRptNum_IsEmpty_Existed'] == 'Yes']
join_1_x

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,FormRptNum_IsEmpty_Existed
0,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,1,NaN,NaN,CodedValue,HEME_PRTRTSET,SCR,CM_PST,HEME_PRTRTSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,DYN_CM_PST_SET_PSTTYOTH_VISIBLE,13,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


#### Output reports: EC_InternalQC_Check_IfFormRptNumIsEntered

In [587]:
join_1_x.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_Check_IfFormRptNumIsEntered.xlsx',index=False)

#### Check If FormRptNum Is Wrongly Entered

In [ ]:
pd.set_option('display.max_rows', 1)
pd.set_option('display.max_columns', 30)

In [588]:
join_1_y = join_1[~join_1['FormOID_y'].str.contains(r'[@#&$%+-/|*]')]
join_1_y

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y
13,DYN_CM_PST_SET_REASENDO_VISIBLE,1,NaN,NaN,CodedValue,REASEND,NaN,CM_PST,REASEND,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,CM_PST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST


In [589]:
def FormRptNum_WronglyEntered(x):
    if pd.isna(x):
        result = 0
    elif ~pd.isna(x):
        result = 1
    else:
        result = 0
    return result

join_1_y['FormRptNum_WronglyEntered'] = join_1_y['FormRepeatNumber'].apply(FormRptNum_WronglyEntered)
join_1_y

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y,FormRptNum_WronglyEntered
13,DYN_CM_PST_SET_REASENDO_VISIBLE,1,NaN,NaN,CodedValue,REASEND,NaN,CM_PST,REASEND,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,CM_PST,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST,0


In [590]:
FormRptNum_WronglyEntered_Sum = join_1_y.groupby('CheckName').sum()
FormRptNum_WronglyEntered_Sum

,StepOrdinal,RecordPosition,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormRptNum_WronglyEntered
CheckName,,,,,,,
Chk_AE_MIX_AEACNOTH_CM1_01_a_CF,3,0.0,0.0,0.0,1,2,0
...,...,...,...,...,...,...,...
Derive_MinDate_Z_CMEDAT4_b_CF,3,0.0,0.0,0.0,1,2,0


In [592]:
FormRptNum_WronglyEntered_Sum = pd.DataFrame(FormRptNum_WronglyEntered_Sum,columns=['FormRptNum_WronglyEntered'])
FormRptNum_WronglyEntered_Sum

,FormRptNum_WronglyEntered
CheckName,
Chk_AE_MIX_AEACNOTH_CM1_01_a_CF,0
...,...
Derive_MinDate_Z_CMEDAT4_b_CF,0


In [593]:
FormRptNum_WronglyEntered_Sum = FormRptNum_WronglyEntered_Sum.rename(columns={'FormRptNum_WronglyEntered':'FormRptNum_WronglyEntered_Sum'})
FormRptNum_WronglyEntered_Sum

,FormRptNum_WronglyEntered_Sum
CheckName,
Chk_AE_MIX_AEACNOTH_CM1_01_a_CF,0
...,...
Derive_MinDate_Z_CMEDAT4_b_CF,0


In [594]:
join_1_y = pd.merge(join_1_y,FormRptNum_WronglyEntered_Sum,how='left',on='CheckName')
join_1_y

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y,FormRptNum_WronglyEntered,FormRptNum_WronglyEntered_Sum
0,DYN_CM_PST_SET_REASENDO_VISIBLE,1,NaN,NaN,CodedValue,REASEND,NaN,CM_PST,REASEND,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,CM_PST,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST,0,0


In [595]:
def Is_EmptyFormRptNum_WronglyEntered(x):
    if x > 0:
        result = 'Yes'
    elif x == 0:
        result = 'No'
    else:
        result = 'n/a'
    return result

join_1_y['Is_EmptyFormRptNum_WronglyEntered'] = join_1_y['FormRptNum_WronglyEntered_Sum'].apply(Is_EmptyFormRptNum_WronglyEntered)
join_1_y

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_Form_Repeated,Is_Form_Repeated_Sum,FormOID_y,FormRptNum_WronglyEntered,FormRptNum_WronglyEntered_Sum,Is_EmptyFormRptNum_WronglyEntered
0,DYN_CM_PST_SET_REASENDO_VISIBLE,1,NaN,NaN,CodedValue,REASEND,NaN,CM_PST,REASEND,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,CM_PST,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,CM_POST,0,0,No


In [597]:
join_1_y = join_1_y.drop(columns=['Is_Form_Repeated','Is_Form_Repeated_Sum','FormOID_y','FormRptNum_WronglyEntered','FormRptNum_WronglyEntered_Sum'])
join_1_y

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_EmptyFormRptNum_WronglyEntered
0,DYN_CM_PST_SET_REASENDO_VISIBLE,1,NaN,NaN,CodedValue,REASEND,NaN,CM_PST,REASEND,0.0,NaN,NaN,NaN,NaN,NaN,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,Chk_CM_POST_CMPDDAT4_02_b_CF,2,IsPresent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [598]:
join_1_y = join_1_y[join_1_y['Is_EmptyFormRptNum_WronglyEntered'] == 'Yes']
join_1_y

,CheckName,StepOrdinal,CheckFunction,StaticValue,DataFormat,VariableOID,FolderOID,FormOID_x,FieldOID,RecordPosition,CustomFunction,LogicalRecordPosition,Scope,OrderBy,FormRepeatNumber,FolderRepeatNumber,Is_EmptyFormRptNum_WronglyEntered
366,Chk_CM_PST_PDDAT_01,1,NaN,NaN,CodedValue,BESTRPH,SCR,CM_PST,BESTRPH,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Chk_CM_PST_PDDAT_01,10,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


##### Output report: EC_InternalQC_Check_IfFormRptNumIsWronglyEntered

In [599]:
join_1_y.to_excel(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_Check_IfFormRptNumIsWronglyEntered.xlsx',index=False)

# Formatting output reports

In [600]:
import openpyxl
#from openpyxl import load_workbook
#from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.styles.colors import Color
#from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
#from openpyxl.utils import get_column_letter
#from openpyxl.styles import PatternFill
#from openpyxl.styles.borders import Border, Side
#from openpyxl.workbook import Workbook
import string

wb1 = openpyxl.load_workbook(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_Check_IfFormRptNumIsEntered.xlsx')
#wb1 = openpyxl.load_workbook(address + 'DefaultValueVsDEntries.xlsx')

wb2 = openpyxl.load_workbook(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_Check_IfFormRptNumIsWronglyEntered.xlsx')

# body_font
GLOBAL_TITLE_FONT = Font(name='Arial',
                         size=10,
                         bold=False,
                         italic=False,
                         vertAlign=None,
                         underline='none',
                         strike=False)
# title font
GLOBAL_TITLE_FONT1 = Font(name='Arial',
                          size=10,
                          bold=True,
                          italic=False,
                          vertAlign=None,
                          underline='none',
                          strike=False)

BORDER = Border(left=Side(border_style='thin',
                           color='00969696'),
                 right=Side(border_style='thin',
                            color='00969696'),
                 top=Side(border_style='thin',
                          color='00969696'),
                 bottom=Side(border_style='thin',
                             color='00969696'),
                 diagonal=Side(border_style='thin',
                               color='00969696'),
                 diagonal_direction=0,
                 outline=Side(border_style='thin',
                              color='00969696'),
                 vertical=Side(border_style='thin',
                               color='00969696'),
                 horizontal=Side(border_style='thin',
                                color='00969696')
                )

ALIGNHMENT = Alignment(horizontal='general',
                       vertical='top',
                       text_rotation=0,
                       wrap_text=False,
                       shrink_to_fit=False,
                       indent=0)

ALIGNHMENT_HEADER = Alignment(horizontal='center',
                       vertical='top',
                       text_rotation=0,
                       wrap_text=False,
                       shrink_to_fit=False,
                       indent=0)
# color
highlight = PatternFill(start_color='0033CCCC',
                   end_color='0033CCCC',
                   fill_type='solid')

ws1 = wb1['Sheet1']
ws2 = wb2['Sheet1']

#for row in ws.iter_rows(row_offset==1):
    #for cell in row:
        #cell.font = GLOBAL_TITLE_FONT

# assign formatting to cells in each row except for header row
def body_font(ws):
    for i, row in enumerate(ws.iter_rows()):                              
        if i == 0:                                                                              
            continue                                                            
        for cell in row:                                                  
            cell.font = GLOBAL_TITLE_FONT
            cell.border = BORDER
            cell.alignment = ALIGNHMENT

# assign formatting to header row(bolder, border, background color)
def header_font(ws):
    for row in ws.iter_rows(min_row=1, max_row=1):
        for cell in row:
            cell.font = GLOBAL_TITLE_FONT1
            cell.border = BORDER
            cell.fill = PatternFill("solid", start_color="00FFFF99")
            cell.alignment = ALIGNHMENT_HEADER

# auto-width
def auto_width(ws):
    for column_cells in ws.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        ws.column_dimensions[column_cells[0].column_letter].width = length            
# manually-width
def change_width(ws):
    ws.column_dimensions['A'].width = 35
    ws.column_dimensions['F'].width = 15
    ws.column_dimensions['I'].width = 15
    ws.column_dimensions['K'].width = 5
    ws.column_dimensions['L'].width = 5
    ws.column_dimensions['M'].width = 5
    ws.column_dimensions['N'].width = 5
    ws.column_dimensions['O'].width = 15    
    ws.column_dimensions['P'].width = 5
    
def wrap_text(ws):
    ws.cell('D1').style.alignment.wrap_text = True
    ws.column_dimensions['E'].alignment.wrap_text == True

def change_color(ws):
    ws['O1'].fill = highlight
    
# invoke functions
body_font(ws1)        
header_font(ws1)        
auto_width(ws1)
change_width(ws1)
change_color(ws1)
#wrap_text(ws1)

body_font(ws2)        
header_font(ws2)        
auto_width(ws2)
change_width(ws2)
change_color(ws2)
#wrap_text(ws1)

# change width of column B        
#ws1.column_dimensions['B'].width = 25
#ws2.column_dimensions['B'].width = 25
               
wb1.save(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_Check_IfFormRptNumIsEntered.xlsx')
#wb1.save(address + 'DefaultValueVsDEntries.xlsx')

wb2.save(r'C:\Users\wei4.sun\OneDrive - BeiGene\文档\studies\BGB-16673-102\ALS\20211231\EC_InternalQC_Check_IfFormRptNumIsWronglyEntered.xlsx')
#wb1.save(address + 'DefaultValueVsDEntries.xlsx')

#load_workbook.font = GLOBAL_TITLE_FONT